In [1]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers

c:\Users\adity\anaconda3\envs\mchatbot\lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()

    return documents

In [3]:
ext_data = load_pdf("Data/")

In [4]:
def text_split(ext_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=20)
    text_chunks = text_splitter.split_documents(ext_data)

    return text_chunks

In [77]:
text_chunks = text_split(ext_data)

In [6]:
print(len(text_chunks))

7020


In [31]:
# #download embedding model
# def download_hugging_face_embeddings():
#     embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
#     return embeddings
# embeddings = download_hugging_face_embeddings()

In [24]:
from dotenv import load_dotenv
load_dotenv()
import os

In [27]:
PINECONE_API_KEY = os.environ.get('API_KEY')

In [28]:
print(PINECONE_API_KEY)

55f7cc69-3341-48ef-a6b2-708c280aaeea


In [91]:
from pinecone import Pinecone

pc = Pinecone(api_key=PINECONE_API_KEY)
index = pc.Index("medicalproject")

In [95]:
from sentence_transformers import SentenceTransformer
import numpy as np

# Function to download the Hugging Face embedding model
def download_hugging_face_embeddings():
    embeddings = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

# Load the Hugging Face embedding model
model = download_hugging_face_embeddings()

# Extract the data from text chunks

cleaned_data = [t.page_content for t in text_chunks][:1000]

# Generate embeddings for each text chunk
embeddings = model.encode(cleaned_data)
# embeddings = np.array(embeddings)

# Prepare the data for upsert
data = [(str(i), embedding.tolist()) for i, embedding in enumerate(embeddings)]

# Upsert the embeddings into the index
index.upsert(vectors=data)

{'upserted_count': 1000}